In [1]:
import numpy as np
import qcelemental as qcel
import molsym
np.set_printoptions(suppress=True, precision=3, linewidth=600)

In [2]:
# Create MolSym Molecule from xyz file

fn = "test/xyz/ammonia.xyz"
#fn = "test/xyz/methane.xyz"
#fn = "test/xyz/benzene.xyz"

mol = molsym.Molecule.from_file(fn)
print(mol)

pg, (paxis, saxis) = molsym.find_point_group(mol)
print(f"Point Group is {pg}")

MolSym Molecule:
   N       0.00000000     0.00000000     0.22175369
   H       0.00000000     1.76245118    -0.51742402
   H       1.52632801    -0.88122654    -0.51742402
   H      -1.52632801    -0.88122654    -0.51742402

Point Group is C3v


In [3]:
# Create Molecule from Schema

with open(fn, "r") as lfn:
    file_str = lfn.read()

schema = qcel.models.Molecule.from_data(file_str).dict()
mol = molsym.Molecule.from_schema(schema)

pg, (paxis, saxis) = molsym.find_point_group(mol)

print(f"Point Group is {pg}")
print(f"Primary axis for orienting molecule: {paxis}")
print(f"Secondary axis: {saxis}")

Point Group is C3v
Primary axis for orienting molecule: [0. 0. 1.]
Secondary axis: [-0.866 -0.5    0.   ]


In [4]:
# The coordinates in test/xyz/ammonia.xyz are not symmetric to a tight tolerance

print(f"The default tolerance for detecting symmetry is: {mol.tol}")

# Increase tolerance
mol.tol = 1e-12
pg, (paxis, saxis) = molsym.find_point_group(mol)
print(f"The point group for a tolerance of {mol.tol} is {pg}")
mol.tol = 1e-5

# molsym.symmetrize uses a loose tolerance to detect a potential point group, 
# then forces the Molecule to that symmetry to near machine precision
mol = molsym.symmetrize(mol)
pg, (paxis, saxis) = molsym.find_point_group(mol)
print(f"The point group for a tolerance of {mol.tol} is {pg} after symmetrization")

The default tolerance for detecting symmetry is: 1e-05
The point group for a tolerance of 1e-12 is Cs
The point group for a tolerance of 1e-12 is C3v after symmetrization


In [5]:
# Given a valid Schönflies symbol as a string, the symmetry elements (symels) can be generated
molsym.symtext.main.pg_to_symels(pg)

[
 Symbol:          E: [[ 1.00000  0.00000  0.00000],[ 0.00000  1.00000  0.00000],[ 0.00000  0.00000  1.00000]],
 
 Symbol:      C_3^1: [[-0.50000 -0.86603  0.00000],[ 0.86603 -0.50000  0.00000],[ 0.00000  0.00000  1.00000]],
 
 Symbol:      C_3^2: [[-0.50000  0.86603  0.00000],[-0.86603 -0.50000  0.00000],[ 0.00000  0.00000  1.00000]],
 
 Symbol: sigma_v(1): [[ 1.00000  0.00000 -0.00000],[ 0.00000 -1.00000  0.00000],[-0.00000  0.00000  1.00000]],
 
 Symbol: sigma_v(2): [[-0.50000 -0.86603  0.00000],[-0.86603  0.50000  0.00000],[ 0.00000  0.00000  1.00000]],
 
 Symbol: sigma_v(3): [[-0.50000  0.86603  0.00000],[ 0.86603  0.50000 -0.00000],[ 0.00000 -0.00000  1.00000]]]

In [6]:
# Similarly, the real valued character table of any point group can be generated
molsym.symtext.main.pg_to_chartab(pg)

Character Table for C3v
Irreps: ['A1', 'A2', 'E']
Classes: ['E', '2C_3', '3sigma_v']
Characters:
[[ 1.  1.  1.]
 [ 1.  1. -1.]
 [ 2. -1.  0.]]

In [7]:
"""
Most of the necessary symmetry information is stored in the Symtext object, such as:
    - A molecule rotated to the symmetry element reference frame
    - Matrices for rotating Cartesian vectors to and from the symmetry element ref. frame
    - List of symmetry elements
    - Character table object
    - Map of all atoms in the Molecule under each symmetry operation
    - Multiplication table of the symmetry operations
"""

symtext = molsym.Symtext.from_molecule(mol)
print(symtext)


MolSym Molecule:
   N      -0.00000000     0.00000000     0.13125886
   H      -0.88122565    -1.52632759    -0.60791885
   H      -0.88122565     1.52632759    -0.60791885
   H       1.76245129    -0.00000000    -0.60791885

Character Table for C3v
Irreps: ['A1', 'A2', 'E']
Classes: ['E', '2C_3', '3sigma_v']
Characters:
[[ 1.  1.  1.]
 [ 1.  1. -1.]
 [ 2. -1.  0.]]

[
Symbol:          E: [[ 1.00000  0.00000  0.00000],[ 0.00000  1.00000  0.00000],[ 0.00000  0.00000  1.00000]], 
Symbol:      C_3^1: [[-0.50000 -0.86603  0.00000],[ 0.86603 -0.50000  0.00000],[ 0.00000  0.00000  1.00000]], 
Symbol:      C_3^2: [[-0.50000  0.86603  0.00000],[-0.86603 -0.50000  0.00000],[ 0.00000  0.00000  1.00000]], 
Symbol: sigma_v(1): [[ 1.00000  0.00000 -0.00000],[ 0.00000 -1.00000  0.00000],[-0.00000  0.00000  1.00000]], 
Symbol: sigma_v(2): [[-0.50000 -0.86603  0.00000],[-0.86603  0.50000  0.00000],[ 0.00000  0.00000  1.00000]], 
Symbol: sigma_v(3): [[-0.50000  0.86603  0.00000],[ 0.86603  0.50000 -0.

In [8]:
"""
The main goal of MolSym is to return Symmetry Adapted Linear Combinations of arbitrary functions.
Built in are SALCs of Cartesian coordinates, internal coordinates, and atomic basis functions.
"""

cart_coords = molsym.salcs.CartesianCoordinates(symtext)
salcs = molsym.salcs.ProjectionOp(symtext, cart_coords)
print(salcs)

SALC from P^A1_00 (2) gamma= 2.373
[-0.     0.     0.421  0.    -0.    -0.524  0.    -0.    -0.524  0.    -0.    -0.524]
SALC from P^A1_00 (3) gamma= 3.464
[-0.    -0.     0.     0.289  0.5    0.     0.289 -0.5   -0.    -0.577 -0.    -0.   ]
SALC from P^E_00 (0) gamma= 2.695
[ 0.371 -0.    -0.    -0.461  0.    -0.193 -0.461  0.    -0.193 -0.461  0.     0.387]
SALC from P^E_10 (0) gamma= 2.695
[ 0.     0.371  0.    -0.    -0.461 -0.335 -0.    -0.461  0.335 -0.    -0.461  0.   ]
SALC from P^E_00 (3) gamma= 1.838
[-0.315  0.     0.     0.544 -0.265  0.164  0.544  0.265  0.164  0.084  0.    -0.328]
SALC from P^E_10 (3) gamma= 1.838
[-0.    -0.315 -0.    -0.265  0.238  0.284  0.265  0.238 -0.284  0.     0.697 -0.   ]



In [ ]:
B = salcs.basis_transformation_matrix
print(B)

In [9]:
# Ignore how we define internal coordinates, this is subject to change
ic_coords = [[0,1],[0,2],[0,3],[1,0,2],[1,0,3],[2,0,3]]
ic_names = ["R1", "R2", "R3", "A1", "A2", "A3"]
ic_list = [[ic_coords[i], ic_names[i]] for i in range(len(ic_coords))]

# SALCs of internal coordinates
ics = molsym.salcs.InternalCoordinates(ic_list, symtext)
salcs = molsym.salcs.ProjectionOp(symtext, ics)
print(salcs)

SALC from P^A1_00 (0) gamma= 1.732
[0.577 0.577 0.577 0.    0.    0.   ]
SALC from P^A1_00 (3) gamma= 1.732
[0.    0.    0.    0.577 0.577 0.577]
SALC from P^E_00 (0) gamma= 2.449
[ 0.408  0.408 -0.816  0.     0.     0.   ]
SALC from P^E_10 (0) gamma= 2.449
[ 0.707 -0.707  0.     0.     0.     0.   ]
SALC from P^E_00 (3) gamma= 1.225
[ 0.     0.     0.     0.816 -0.408 -0.408]
SALC from P^E_10 (3) gamma= 1.225
[ 0.     0.     0.     0.     0.707 -0.707]

